### In order to predict the values of microbusiness density for the year 2023, the census data for 2022 are required. However, the census data for 2022 is not out yet. In this notebook, the census data of the previous years is used to predict the respective values for the year 2022

In [25]:
import pandas as pd
pd.options.display.max_columns = None

In [26]:
folder = r"C:\Users\edink\Downloads\godaddy-microbusiness-density-forecasting"
ending = ".csv"
census_starter = "\census_starter"

In [27]:
census_starter_df = pd.read_csv(folder+census_starter+ending)
census_starter_df.head()

,pct_bb_2017,pct_bb_2018,pct_bb_2019,pct_bb_2020,pct_bb_2021,cfips,pct_college_2017,pct_college_2018,pct_college_2019,pct_college_2020,pct_college_2021,pct_foreign_born_2017,pct_foreign_born_2018,pct_foreign_born_2019,pct_foreign_born_2020,pct_foreign_born_2021,pct_it_workers_2017,pct_it_workers_2018,pct_it_workers_2019,pct_it_workers_2020,pct_it_workers_2021,median_hh_inc_2017,median_hh_inc_2018,median_hh_inc_2019,median_hh_inc_2020,median_hh_inc_2021
0,76.6,78.9,80.6,82.7,85.5,1001,14.5,15.9,16.1,16.7,16.4,2.1,2.0,2.3,2.3,2.1,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0
1,74.5,78.1,81.8,85.1,87.9,1003,20.4,20.7,21.0,20.2,20.6,3.2,3.4,3.7,3.4,3.5,1.4,1.3,1.4,1.0,1.3,52562,55962.0,58320,61756.0,64346.0
2,57.2,60.4,60.5,64.6,64.6,1005,7.6,7.8,7.6,7.3,6.7,2.7,2.5,2.7,2.6,2.6,0.5,0.3,0.8,1.1,0.8,33368,34186.0,32525,34990.0,36422.0
3,62.0,66.1,69.2,76.1,74.6,1007,8.1,7.6,6.5,7.4,7.9,1.0,1.4,1.5,1.6,1.1,1.2,1.4,1.6,1.7,2.1,43404,45340.0,47542,51721.0,54277.0
4,65.8,68.5,73.0,79.6,81.0,1009,8.7,8.1,8.6,8.9,9.3,4.5,4.4,4.5,4.4,4.5,1.3,1.4,0.9,1.1,0.9,47412,48695.0,49358,48922.0,52830.0


### Backward fill the new missing values in the resampled data

In [30]:
columns = census_starter_df.columns
for col in columns:
    census_starter_df[col] = census_starter_df[col].bfill()

### Preperation steps that enable the prediction of the census data for 2022:
<ol>
   <li>
       Create a list of new columns (pct_bb, pct_colleg, pct_foreign_born, pct_it_workers, median_hh_inc + _2022)
    </li>
    <li>
     Create a dataframe for each census column in the following format:  pct_bb_2017, ..., pct_bb_2021, cfips, etc. and store the created dataframes in a list
    </li>
</ol>

In [31]:
new_cols = ["pct_bb_2022", "pct_college_2022", "pct_foreign_born_2022", "pct_it_workers_2022", "median_hh_inc_2022"]
year_cols = ["year1", "year2", "year3", "year4", "year5", "cfips"]

pct_bb_df = census_starter_df[["pct_bb_2017", "pct_bb_2018", "pct_bb_2019", "pct_bb_2020", "pct_bb_2021", "cfips"]]
pct_college_df = census_starter_df[["pct_college_2017", "pct_college_2018", "pct_college_2019", "pct_college_2020", 
                                    "pct_college_2021", "cfips"]]
pct_foreign_born_df = census_starter_df[["pct_foreign_born_2017", "pct_foreign_born_2018", "pct_foreign_born_2019", 
                                         "pct_foreign_born_2020", "pct_foreign_born_2021", "cfips"]]
pct_it_workers_df = census_starter_df[["pct_it_workers_2017", "pct_it_workers_2018", "pct_it_workers_2019", 
                                       "pct_it_workers_2020", "pct_it_workers_2021", "cfips"]]
median_hh_inc_df = census_starter_df[["median_hh_inc_2017", "median_hh_inc_2018", "median_hh_inc_2019", 
                                      "median_hh_inc_2020", "median_hh_inc_2021", "cfips"]]


In [32]:
df_list = [pct_bb_df, pct_college_df, pct_foreign_born_df, pct_it_workers_df, median_hh_inc_df]

In [33]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
import numpy as np

### For each census column (pct_bb, pct_colleg, pct_foreign_born, pct_it_workers, median_hh_inc) and cfips county code, the census value for 2022 is predicted. Therefore, the following program loops over the list that contains the census dataframes (df_list):
<ol>
   <li>
       For each dataframe in df_list, loop over each unique county code
       <ol>
           <li>
               Retrieve the entry per county code and store the column values in a list. The first entry of this list will server as a building block to create a new column. For example take the dataframe pct_bb_df. The first entry would be pct_bb_2017. A new column with _2022 will be created (pct_bb_2022)
           </li>
           <li>
               Let's say we continue with the dataframe pct_bb_df and retrieve the entry for cfips 1001. This entry contains the values of pct_bb over a span of 5 years (2017-2021 of year1 - year2). The retrieved entry is stored in a dataframe where the column Year contains the years 1-5 and the column Target contains respective values for pct_bb for each year.
           </li>
           <li>
               The previously created dataframe containing the years and target values for each census column per county code serves as a dataset to fit a linear regression model to predict the census column value for the year 2022. The fitted model takes the number 6 (year6 or 2022) to predict the value for 2022. The predicted value is then stored in a dictionary of the following format (resembles a json format): predicted_values_2022{"pct_bb_2022": [ ], ..., "median_hh_inc_2022" [ ] }
           </li>
       </ol>
   </li>
</ol>

In [43]:
cfips = census_starter_df.cfips.unique()
x = [1, 2, 3, 4, 5]
container_df = pd.DataFrame(columns=["Year", "Target"])
predicted_values_2022 = {}
for df in df_list:
    for cfip in cfips:
        y = []
        cfip_entry = df.loc[df.cfips == cfip]
        cfip_columns = cfip_entry.columns
        for col in cfip_columns:
            if col != "cfips":
                y.append(cfip_entry[col].iloc[0])
        container_df["Target"] = y
        container_df["Year"] = x
        reg = LinearRegression().fit(container_df.Year.to_numpy()[:, np.newaxis], container_df.Target.to_numpy()[:, np.newaxis])
        predicted_value_2022 = reg.predict(np.array([[6]]))
        key = df.columns[0][0:-4] + "2022"
        if key in predicted_values_2022:
            predicted_values_2022[key].append(predicted_value_2022[0][0])
        else:
            predicted_values_2022[key] = [predicted_value_2022[0][0]]
        #break
        y.clear()
        container_df = pd.DataFrame(columns=["Year", "Target"])

In [45]:
print(predicted_values_2022.keys())

dict_keys(['pct_bb_2022', 'pct_college_2022', 'pct_foreign_born_2022', 'pct_it_workers_2022', 'median_hh_inc_2022'])


In [50]:
for key in predicted_values_2022:
    print(predicted_values_2022[key])

[87.34, 91.62, 67.15999999999998, 80.16, 86.03, 64.19, 81.27000000000001, 84.5, 81.04000000000002, 79.14000000000001, 84.86000000000001, 67.87, 72.9, 75.51999999999998, 77.28999999999999, 86.51, 76.74000000000001, 69.82000000000002, 82.03999999999999, 79.07999999999998, 69.04, 86.57000000000002, 86.48, 78.71000000000001, 86.42, 87.23, 71.73999999999998, 86.53999999999999, 74.19999999999999, 62.4, 73.99000000000001, 61.279999999999994, 71.88, 83.9, 84.38, 82.75999999999999, 88.7, 68.25, 82.50999999999999, 72.5, 88.11000000000001, 90.05999999999999, 57.78, 74.39000000000001, 91.64, 76.63, 73.85, 90.75999999999999, 82.91, 54.74, 86.96999999999998, 82.27000000000001, 62.88, 76.16, 77.60999999999999, 80.18, 79.45, 89.36, 94.3, 73.25999999999999, 85.64000000000001, 82.6, 87.18, 83.68, 76.63, 77.70999999999998, 81.19999999999999, 53.42, 79.33, 94.32999999999998, 75.73, 76.7, 82.99000000000001, 74.64, 90.15, 87.62000000000002, 54.91999999999999, 93.39000000000001, 87.42, 89.46, 90.130000000000

### For each cfips county code and census colum, the predicted values for 2022 are stored in the variable predicted_values_2022. The keys of the variable resemble the newly created columns and each key posesses a list of values which resemble the row dimension of the original census dataframe. Each key of the variable predicted_values_2022 is then appended to a copy of the original dataframe.

In [53]:
census_with_2022_pred_df = census_starter_df.copy()
census_with_2022_pred_df["pct_bb_2022"] = predicted_values_2022["pct_bb_2022"]
census_with_2022_pred_df["pct_college_2022"] = predicted_values_2022["pct_college_2022"]
census_with_2022_pred_df["pct_foreign_born_2022"] = predicted_values_2022["pct_foreign_born_2022"]
census_with_2022_pred_df["pct_it_workers_2022"] = predicted_values_2022["pct_it_workers_2022"]
census_with_2022_pred_df["median_hh_inc_2022"] = predicted_values_2022["median_hh_inc_2022"]
census_with_2022_pred_df.head()

,pct_bb_2017,pct_bb_2018,pct_bb_2019,pct_bb_2020,pct_bb_2021,cfips,pct_college_2017,pct_college_2018,pct_college_2019,pct_college_2020,pct_college_2021,pct_foreign_born_2017,pct_foreign_born_2018,pct_foreign_born_2019,pct_foreign_born_2020,pct_foreign_born_2021,pct_it_workers_2017,pct_it_workers_2018,pct_it_workers_2019,pct_it_workers_2020,pct_it_workers_2021,median_hh_inc_2017,median_hh_inc_2018,median_hh_inc_2019,median_hh_inc_2020,median_hh_inc_2021,pct_bb_2022,pct_college_2022,pct_foreign_born_2022,pct_it_workers_2022,median_hh_inc_2022
0,76.6,78.9,80.6,82.7,85.5,1001,14.5,15.9,16.1,16.7,16.4,2.1,2.0,2.3,2.3,2.1,1.3,1.1,0.7,0.6,1.1,55317,58786.0,58731,57982.0,62660.0,87.34,17.30,2.25,0.69,62859.8
1,74.5,78.1,81.8,85.1,87.9,1003,20.4,20.7,21.0,20.2,20.6,3.2,3.4,3.7,3.4,3.5,1.4,1.3,1.4,1.0,1.3,52562,55962.0,58320,61756.0,64346.0,91.62,20.55,3.62,1.13,67397.8
2,57.2,60.4,60.5,64.6,64.6,1005,7.6,7.8,7.6,7.3,6.7,2.7,2.5,2.7,2.6,2.6,0.5,0.3,0.8,1.1,0.8,33368,34186.0,32525,34990.0,36422.0,67.16,6.71,2.59,1.12,36371.8
3,62.0,66.1,69.2,76.1,74.6,1007,8.1,7.6,6.5,7.4,7.9,1.0,1.4,1.5,1.6,1.1,1.2,1.4,1.6,1.7,2.1,43404,45340.0,47542,51721.0,54277.0,80.16,7.32,1.44,2.23,56894.9
4,65.8,68.5,73.0,79.6,81.0,1009,8.7,8.1,8.6,8.9,9.3,4.5,4.4,4.5,4.4,4.5,1.3,1.4,0.9,1.1,0.9,47412,48695.0,49358,48922.0,52830.0,86.03,9.32,4.46,0.79,52762.3


In [54]:
census_with_2022_pred_df.to_csv("census_with_2022_predicted.csv")